In [65]:
import numpy as np
import sys
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [55]:
def encode_symbols(symbols):
    # Make unique characters
    symbols = set(symbols)
    # Sort for logical ordering scheme
    symbols = sorted(symbols)
    sym_to_code = {sym:code for code,sym in enumerate(symbols)}
    code_to_sym = {code:sym for code,sym in enumerate(symbols)}
    return sym_to_code, code_to_sym

In [56]:
text = open('sherlock.txt', 'r').read()
# Make text smaller for testing
text = text[0:100000]
# Better to use word2vec
char_to_code, code_to_char = encode_symbols(text)
# Constants
vocab_size = len(char_to_code)
n_hidden = 256
n_epochs = 1
seq_length = 100

In [77]:
# Creating train set
dataX = []
dataY = []

for i in range(len(text) - seq_length):
    seq_in = text[i:i+seq_length]
    seq_out = text[i+seq_length]
    dataX.append([char_to_code[ch] for ch in seq_in])
    dataY.append(char_to_code[seq_out])
    
n_train_seq = len(dataX)

In [58]:
# Reshape into [samples, train_seq, features]
x = np.reshape(dataX, (n_train_seq, seq_length, 1))
# Normalize
x = x / float(vocab_size)
# y into one hot 
y = np_utils.to_categorical(dataY)

In [59]:
# LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(x.shape[1], x.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [60]:
# Define the checkpoints
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [45]:
# fit the model
model.fit(x, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
 7680/99900 [=>............................] - ETA: 4:10 - loss: 3.3302

KeyboardInterrupt: 

In [46]:
# Load Trained Weights
model.load_weights('filename')

ImportError: `load_weights` requires h5py.

In [69]:
def sample_text(sample_length):
    # Random Seed
    pattern = dataX[np.random.randint(0, len(dataX) - 1)]
    print('Seed: ')
    print(''.join([code_to_char[ch] for ch in pattern]))
    print('---------------------------------')
    for i in range(sample_length):
        x = np.reshape(pattern, (1, len(pattern), 1))
        x = x / float(vocab_size)
        pred = model.predict(x, verbose=0)
        index = np.argmax(pred)
        result = code_to_char[np.argmax(pred)]
        sys.stdout.write(result)
        pattern.append(index)
        pattern = pattern[1:len(pattern)]

In [73]:
sample_text(10)

Seed: 
hum! Prima donna Imperial Opera
of Warsaw--yes! Retired from operatic stage--ha! Living in
London--q
---------------------------------
uuuuuuuuuu